# Stage 09 — Feature Engineering Lecture Notebook

In this notebook, we demonstrate simple but powerful feature engineering transformations on a synthetic financial dataset.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures

# Generate synthetic financial dataset
np.random.seed(42)
n = 200

df = pd.DataFrame({
    'customer_id': range(1, n+1),
    'income': np.random.normal(60000, 15000, n).astype(int),
    'credit_score': np.random.normal(680, 50, n).astype(int),
    'monthly_spend': np.random.normal(2000, 600, n).astype(int),
    'region': np.random.choice(['North','South','East','West'], n),
    'default_flag': np.random.choice([0,1], n, p=[0.8,0.2])
})
df.head()

## Basic Transformations

In [ ]:
# Percent change in monthly spending
df['spend_pct_change'] = df['monthly_spend'].pct_change().fillna(0)

# Rolling mean of spending (3-month window)
df['rolling_spend_mean'] = df['monthly_spend'].rolling(3).mean().fillna(df['monthly_spend'])

# Binary flag for high spenders
df['high_spender_flag'] = (df['monthly_spend'] > 2500).astype(int)
df.head()

## Categorical Encoding

In [ ]:
df.head(3)

In [ ]:
# One-hot encoding
df_onehot = pd.get_dummies(df, columns=['region'])
df_onehot 

In [ ]:
# Label encoding
le = LabelEncoder()
df['region_label'] = le.fit_transform(df['region'])
df

In [ ]:
# Frequency encoding
region_freq = df['region'].value_counts(normalize=True)
df['region_freq'] = df['region'].map(region_freq)
df[['region','region_label','region_freq']].head()

## Interaction & Polynomial Features

In [ ]:
# Interaction feature
df['income_x_credit'] = df['income'] * df['credit_score']

# Squared feature
df['income_squared'] = df['income'] ** 2

# Polynomial features using sklearn
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_features = poly.fit_transform(df[['income','credit_score']])
poly_df = pd.DataFrame(poly_features, columns=poly.get_feature_names_out(['income','credit_score']))
df = pd.concat([df, poly_df], axis=1)
df.head()

## Group Walkthrough Implementation

In [ ]:
def create_features(df):
    """
    Create engineered features for modeling.
    Assumptions:
    - Rolling mean of spending captures short-term trends.
    - Spend-to-income ratio helps normalize across different income levels.
    """
    df['spend_income_ratio'] = df['monthly_spend'] / df['income']
    df['rolling_spend_mean'] = df['monthly_spend'].rolling(3).mean().fillna(df['monthly_spend'])
    return df

# df = create_features(df)
df.head()

## Feature Engineering Additional Examples
Feature engineering can be roughly categorized into 10 types:

1. **Combining columns to make a feature** – create new features by combining existing ones.
2. **Splitting a column into multiple features** – extract meaningful components from a single feature.
3. **Transforming features point by point** – apply math or functions to each value individually.
4. **Transforming features by window** – rolling averages, cumulative sums, or other window-based transforms.
5. **Splitting continuous features into categories** – bin numeric features into discrete bins.
6. **Aggregating features across groups** – compute group-level statistics like mean, median, etc.
7. **Interactions between features** – multiply, divide, or create polynomial combinations of features.
8. **Temporal or sequential features** – time since events, day-of-week, seasonal indicators, or differences.
9. **Encoding external knowledge** – map raw features to domain-specific insights (e.g., zip→region, text→sentiment).
10. **Feature selection / reduction** – reduce dimensionality or remove uninformative features (e.g., PCA, factor analysis).

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

# Synthetic dataset
np.random.seed(42)
n = 20
df = pd.DataFrame({
    'column0': np.random.randint(1, 10, size=n),
    'column1': np.random.randint(10, 50, size=n)
})

# 1. Combining columns
df['sum_col'] = df['column0'] + df['column1']

# 2. Splitting a column (simulate time split)
df['column1_div_10'] = df['column1'] // 10
df['column1_mod_10'] = df['column1'] % 10

# 3. Transforming point by point
df['log_col1'] = np.log1p(df['column1'])

# 4. Transforming by window
df['rolling_col1'] = df['column1'].rolling(3, min_periods=1).mean()

# 5. Splitting continuous into categories
df['col1_bin'] = pd.cut(df['column1'], bins=[10,20,30,40,50], labels=False)

# 6. Aggregating across groups (using binned column)
df['col1_bin_mean'] = df.groupby('col1_bin')['column1'].transform('mean')

# 7. Interactions
df['interaction'] = df['column0'] * df['column1']

# 8. Temporal/sequential (simulated as index differences)
df['diff_col1'] = df['column1'].diff().fillna(0)

# 9. Encoding external knowledge (simulated mapping)
mapping = {0:'A', 1:'B', 2:'C', 3:'D'}
df['col1_category'] = df['col1_bin'].map(mapping)

# 10. Feature reduction (PCA on column0 and column1)
pca = PCA(n_components=1)
df['pca_component'] = pca.fit_transform(df[['column0','column1']])

df

## Rolling Average Comparison Demo (Live Demo Friendly)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# -----------------------------
# Configurable variables
# -----------------------------
roll_var = 'var2'       # Variable to compute rolling average on
sort_var1 = 'date'      # First sort option (time-based)
sort_var2 = 'var1'      # Second sort option (value-based)
window_size = 5         # Rolling window size (number of observations)

# -----------------------------
# Create sample dataset
# -----------------------------
np.random.seed(42)
n = 50
dates = pd.date_range('2025-01-01', periods=n, freq='D')
var1 = np.random.randint(10, 100, size=n)
var2 = np.random.normal(loc=50, scale=10, size=n)

df = pd.DataFrame({'date': dates, 'var1': var1, 'var2': var2})

# -----------------------------
# Rolling averages
# -----------------------------
df_time = df.sort_values(sort_var1).copy()
df_time[f'{roll_var}_rolling_time'] = df_time[roll_var].rolling(window=window_size, min_periods=1).mean()

df_var1 = df.sort_values(sort_var2).copy()
df_var1[f'{roll_var}_rolling_var1'] = df_var1[roll_var].rolling(window=window_size, min_periods=1).mean()

# -----------------------------
# Side-by-side visualization
# -----------------------------
fig, axes = plt.subplots(1, 2, figsize=(14,5))

# First subplot: rolling by time
axes[0].plot(df_time['date'], df_time[roll_var], 'o', alpha=0.3, label=f'Original {roll_var}')
axes[0].plot(df_time['date'], df_time[f'{roll_var}_rolling_time'], color='red', linewidth=2, label=f'Rolling by time (window={window_size})')
axes[0].set_title('Rolling Average by Time Order')
axes[0].set_xlabel('Date')
axes[0].set_ylabel(roll_var)
axes[0].legend()
axes[0].tick_params(axis='x', rotation=45)

# Second subplot: rolling by var1 order, x-axis is var1
axes[1].plot(df_var1[sort_var2], df_var1[roll_var], 'o', alpha=0.3, label=f'Original {roll_var}')
axes[1].plot(df_var1[sort_var2], df_var1[f'{roll_var}_rolling_var1'], color='darkgreen', linewidth=2, label=f'Rolling by {sort_var2} order (window={window_size})')
axes[1].set_title(f'Rolling Average by {sort_var2} Order')
axes[1].set_xlabel(sort_var2)
axes[1].set_ylabel(roll_var)
axes[1].legend()
axes[1].tick_params(axis='x', rotation=0)  # numeric x-axis, no rotation needed

plt.tight_layout()
plt.show()


# -----------------------------
# Combined visualization (green points with dotted line by time)
# -----------------------------
plt.figure(figsize=(10,5))

# Original values (plotted by date)
plt.plot(df_time['date'], df_time[roll_var], 'o', alpha=0.4, label=f'Original {roll_var}')

# Rolling by time (red line)
plt.plot(df_time['date'], df_time[f'{roll_var}_rolling_time'], color='red', linewidth=2, label=f'Rolling by time (window={window_size})')

# Rolling by var1 order, connect by date
df_var1_sorted_by_date = df_var1.sort_values('date')
plt.scatter(df_var1_sorted_by_date['date'], df_var1_sorted_by_date[f'{roll_var}_rolling_var1'], color='green', label=f'Rolling by {sort_var2} order (window={window_size})')
plt.plot(df_var1_sorted_by_date['date'], df_var1_sorted_by_date[f'{roll_var}_rolling_var1'], color='green', linestyle=':', alpha=0.6)

plt.title('Rolling Averages Comparison (Combined)')
plt.xlabel('Date')
plt.ylabel(roll_var)
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



# -----------------------------
# Explanation:
# -----------------------------
# - Rolling by time shows trends of the selected variable over chronological order.
# - Rolling by sort_var2 shows trends when ordering by another variable (magnitude-based).
# - Side-by-side plots and combined plot let you compare these smoothing methods.
# - Changing roll_var, sort_var1, sort_var2, or window_size at the top updates all plots immediately.